# Albums downloading

Last step of data downloading was to fetch albums of artists from both Spotify and Musicbrainz datasets. Main reason for that was to make it possible to match artists from both databases using similarity in albums' names and dates (more on this topic in 'data merging' part here: TODO - add link). Secondary reason was to get more insights in the artists data.

In [1]:
import spotipy
import spotipy.oauth2 as oauth2
import spotipy.util as util
from IPython.display import clear_output
from pymongo import MongoClient, errors
import time
import musicbrainzngs
import sys
from datapackage import Package
import psycopg2

In [2]:
SPOTIPY_CLIENT_ID = 'd70e66cbc22e4e1287c50c97b54c2457'
SPOTIPY_CLIENT_SECRET = 'e20d58cae494486392723da75a3ef393'
SPOTIPY_REDIRECT_URI = 'http://localhost/?code=...'

credentials = oauth2.SpotifyClientCredentials(
client_id="d70e66cbc22e4e1287c50c97b54c2457",
client_secret="e20d58cae494486392723da75a3ef393")

token = credentials.get_access_token()
print(token)
spotify = spotipy.Spotify(auth=token)

db_client = MongoClient('localhost', 27017)
db = db_client.musicdata

BQC7q92MBvmm2IELx_JT0i8LRbgPIDOUT1tDXZIBcbf9cprLMrHMekFwYk2qPr5SExKcB7PYEKq6YWN0xpw


## Download albums from Spotify

For Spotify dataset only artists with popularity graten than zero were taken into account while downloading albums. The reason for that was that this process was very time-consuming and at this point preliminary decision was made to do the analysis and classification part using only data for more popular artists.

In [3]:
def fetch_albums(_id, offset,result):
    keys = ['album_group', 'album_type','artists', 'available-markets', 'id','name', 'release_date', 'release_date_precision', 'type']
    fetched = spotify.artist_albums(_id, limit=50, offset = offset)

    items = fetched['items']
    albums =[]
    for it in items:
        album = {k:v for k, v in it.items() if k in keys}
        albums.append(album)
    
    result.append(albums)
    
    if fetched['next'] != None:
        return fetch_albums(_id, offset+50, result)
    
    return result

In [34]:
artist_collection = db.get_collection("artists_spotify_bynames")
albums_collection = db.get_collection("albums_spotify")
keys = ['album_group', 'album_type','artists', 'available-markets', 'id','name', 'release_date', 'release_date_precision', 'type']
num = 0
not_fetched = []

for artist in artist_collection.find({'popularity': {'$gt': 0}},no_cursor_timeout=True):
    clear_output(wait=True)
    print(num)
    num +=1
    _id = artist['_id']
    try:
        albums = fetch_albums(_id,0, [])
    except spotipy.client.SpotifyException as err:
        if err.code == 401:
            print("Spotipy 401 error:", err.msg)
            not_fetched.append(_id)
            time.sleep(60)
            continue
        else:
            print("Spotipy error:", err.code, err.http_status, err.msg)
            token = credentials.get_access_token()
            print(token)
            spotify = spotipy.Spotify(auth=token)
            not_fetched.append(_id)
            continue
    except Exception as err:
        print("Exception ", err)
        time.sleep(20)
        not_fetched.append(_id)
        continue
        
    artist_albums = {'_id':_id, 'albums':albums }
    albums_collection.update_one({'_id':artist_albums['_id']}, {"$set": artist_albums}, upsert=True)
    #print(artist_albums)



487387


In [37]:
#try again to download albums for artists that were not fetched before due to some errors
print(len(not_fetched))
for id in not_fetched:
    try:
        albums = fetch_albums(_id,0, [])
    except spotipy.client.SpotifyException as err:
        if err.code == 401:
            print("Spotipy 401 error:", err.msg)
            not_fetched.append(_id)
            time.sleep(60)
            continue
        else:
            print("Spotipy error:", err.code, err.http_status, err.msg)
            token = credentials.get_access_token()
            print(token)
            spotify = spotipy.Spotify(auth=token)
            not_fetched.append(_id)
            continue
    except Exception as err:
        print("Exception ", err)
        time.sleep(20)
        not_fetched.append(_id)
        continue
        
    artist_albums = {'_id':_id, 'albums':albums }
    albums_collection.update_one({'_id':artist_albums['_id']}, {"$set": artist_albums}, upsert=True) 

61


## Download albums from Musicbrainz

Since this data was downloaded from local database, time was not a problem as in previous case. This is why all albums for all artists were fetched.

In [4]:
# Connect to an existing database
conn = psycopg2.connect(dbname="musicbrainz_db",user='musicbrainz', host="localhost",password="musicbrainz")
   

# Open a cursor to perform database operations
cur = conn.cursor()

#dictionary for release type
release_type = {}

cur.execute("SELECT id, name FROM release_group_primary_type")
types = cur.fetchall()
for t in types:
    release_type[t[0]] = t[1]

In [5]:
print(release_type)

{1: 'Album', 2: 'Single', 3: 'EP', 12: 'Broadcast', 11: 'Other'}


In [6]:
alb_collection = db.get_collection("albums_musicbrainz")

In [7]:
mb_artists = db.get_collection("artists_mb")
artists = {}
for artist in mb_artists.find():
    artists[artist['db_id']] = artist['_id']

    
print(len(artists))
not_found = []

1318077


In [8]:
print(artists.get(805192))

8972b1c1-6482-4750-b51f-596d2edea8b1


In [10]:
cur.execute("SELECT RG.id, RG.name, RG.artist_credit, RG.type, RGM.first_release_date_year, RGM.rating FROM release_group RG INNER JOIN release_group_meta RGM ON RG.id = RGM.id")
num = 0

for album in cur:
    artist_id = artists.get(album[2])
       
    album ={
        '_id': album[0],
        'name': album[1],
        'artist_id': artist_id,
        'type': release_type.get(album[3], 'unknown'),
        'release_year': album[4],
        'rating': album[5]
    }

    alb_collection.insert_one(album)
